<a href="https://colab.research.google.com/github/giopnd/notebooks/blob/master/text_rnn0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv

# check english lexicon
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import (
    wordnet,
    stopwords
)

# handle regular expressions
import re

# handle data
import pandas as pd
import numpy as np

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, Activation;
import tensorflow_datasets as tfds

#import libraries for visualization
import matplotlib.pyplot as plt

print(tf.__version__)

In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "georgiosgiotis"
os.environ['KAGGLE_KEY'] = "78e14d9a6090bb989f7240761e76185b"
# Colab library to upload files to notebook
from google.colab import files
# Install Kaggle library
!pip install -q kaggle
# Downlaod data
!kaggle datasets download -d kazanova/sentiment140
# unzip
!unzip "sentiment140.zip"

In [0]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None)

In [0]:
# data cleaning
def preprocessing_text(df):
  # lowercase
  df = df.str.lower()
  # remove retweets
  df = df.str.replace('rt', '')
  # remove mentions
  df = df.replace(r'@\w+', '', regex=True)
  # remove links
  df = df.replace(r'http\S+', '', regex=True)
  df = df.replace(r'www.[^ ]+', '', regex=True)
  # remove numbers
  df = df.replace(r'[0-9]+', '', regex=True)
  # remove special characters and puntuation marks
  df = df.replace(r'[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
  return df

In [0]:
data0 = data[0].apply(lambda x: int(x)/4)
data5 = data[5]

data5 = preprocessing_text(data5)

# split data into training and test dataset
def split(dfd, dfl):
  x_train, x_test, y_train, y_test = train_test_split(dfd, dfl, test_size=0.2, shuffle=True)
  return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = split(data5[::64], data0[::64])

print(len(x_test))

def tokenize_tweets(dataset, features):
  tokenization = TfidfVectorizer(max_features=features)
  tokenization.fit(dataset)
  dataset_transformed = tokenization.transform(dataset).toarray()
  return dataset_transformed

x_train_mod = tokenize_tweets(x_train, 3500)

In [0]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()
for text_tensor in data5:
  tokenized = re.sub('[,?.]','', text_tensor).lower().split(' ')
  print(tokenized)
  #some_tokens = tokenizer.tokenize(text_tensor.numpy())
  #vocabulary_set.update(some_tokens)
vocab_size = len(vocabulary_set)
vocab_size

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(3500, 64),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              optimizer=tf.keras.optimizers.Adam(1e-4),
#              metrics=['accuracy'])

history = model.fit(x_train_mod, y_train,
                    batch_size=64,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    shuffle=True)